In [4]:
import pandas as pd
import numpy as np

In [5]:
"""
Load data
"""
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

# Handle missing
df_train = df_train.drop(['Name', 'Ticket', 'Cabin'], axis=1).dropna()
df_test = df_test.fillna(method='ffill')

In [6]:
"""
Feature engineering
"""
from sklearn import preprocessing
from keras.utils import to_categorical

le = preprocessing.LabelEncoder()
mms = preprocessing.MinMaxScaler()

# Embarked
embarked_train = to_categorical(le.fit_transform(df_train['Embarked']))
embarked_test = to_categorical(le.fit_transform(df_test['Embarked']))

# Fare
fare_train = df_train['Fare'].values.reshape(-1, 1)
fare_test = df_test['Fare'].values.reshape(-1, 1)

# Siblings
siblings_train = df_train['SibSp'].values.reshape(-1, 1)
siblings_test = df_test['SibSp'].values.reshape(-1, 1)

# Parents
parents_train = df_train['Parch'].values.reshape(-1, 1)
parents_test = df_test['Parch'].values.reshape(-1, 1)

# Sex
sex_train = to_categorical(le.fit_transform(df_train['Sex']))
sex_test = to_categorical(le.fit_transform(df_test['Sex']))

# Pclass
pclass_train = to_categorical(le.fit_transform(df_train['Pclass']))
pclass_test = to_categorical(le.fit_transform(df_test['Pclass']))

# Age
age_train = df_train['Age'].values.reshape(-1, 1)
age_test  = df_test['Age'].values.reshape(-1, 1)

# X and Y
num = len(df_train)
split = int(num * .75)

x_train = np.hstack([pclass_train, sex_train, age_train, siblings_train, parents_train, embarked_train])
x_train = mms.fit_transform(x_train)

y_train = df_train['Survived'].values

# X test
x_test = np.hstack([pclass_test,  sex_test,  age_test, siblings_test, parents_test, embarked_test])
x_test = mms.fit_transform(x_test)

print('Training data:')
[print(", ".join(map(lambda x: str(x), row))) for row in x_train[0:10,:]]

print('Test data:')
[print(", ".join(map(lambda x: str(x), row))) for row in x_test[0:10,:]]

print()

Training data:
0.0, 0.0, 1.0, 0.0, 1.0, 0.271173661724, 0.2, 0.0, 0.0, 0.0, 1.0
1.0, 0.0, 0.0, 1.0, 0.0, 0.472229203317, 0.2, 0.0, 1.0, 0.0, 0.0
0.0, 0.0, 1.0, 1.0, 0.0, 0.321437547122, 0.0, 0.0, 0.0, 0.0, 1.0
1.0, 0.0, 0.0, 1.0, 0.0, 0.434531289269, 0.2, 0.0, 0.0, 0.0, 1.0
0.0, 0.0, 1.0, 0.0, 1.0, 0.434531289269, 0.0, 0.0, 0.0, 0.0, 1.0
1.0, 0.0, 0.0, 0.0, 1.0, 0.673284744911, 0.0, 0.0, 0.0, 0.0, 1.0
0.0, 0.0, 1.0, 0.0, 1.0, 0.0198542347323, 0.6, 0.166666666667, 0.0, 0.0, 1.0
0.0, 0.0, 1.0, 1.0, 0.0, 0.334003518472, 0.0, 0.333333333333, 0.0, 0.0, 1.0
0.0, 1.0, 0.0, 1.0, 0.0, 0.170645890927, 0.2, 0.0, 1.0, 0.0, 0.0
0.0, 0.0, 1.0, 1.0, 0.0, 0.0449861774315, 0.2, 0.166666666667, 0.0, 0.0, 1.0
Test data:
0.0, 0.0, 1.0, 0.0, 1.0, 0.452723196624, 0.0, 0.0, 0.0, 1.0, 0.0
0.0, 0.0, 1.0, 1.0, 0.0, 0.617565607279, 0.125, 0.0, 0.0, 0.0, 1.0
0.0, 1.0, 0.0, 0.0, 1.0, 0.815376500066, 0.0, 0.0, 0.0, 1.0, 0.0
0.0, 0.0, 1.0, 0.0, 1.0, 0.353817750231, 0.0, 0.0, 0.0, 0.0, 1.0
0.0, 0.0, 1.0, 1.0, 0.0, 0.

In [11]:
"""
Define and train model
"""
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

model = Sequential()
model.add(Dense(16, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(16, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(16, input_dim=11))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_train, y_train, validation_split=.1, epochs=200, batch_size=16, shuffle=True)

Train on 640 samples, validate on 72 samples
Epoch 1/200
640/640 [==============================] - 0s - loss: 0.7027 - acc: 0.4547 - val_loss: 0.6722 - val_acc: 0.5972
Epoch 2/200
640/640 [==============================] - 0s - loss: 0.6737 - acc: 0.5906 - val_loss: 0.6527 - val_acc: 0.6389
Epoch 3/200
640/640 [==============================] - 0s - loss: 0.6632 - acc: 0.5938 - val_loss: 0.6415 - val_acc: 0.6250
Epoch 4/200
640/640 [==============================] - 0s - loss: 0.6556 - acc: 0.5891 - val_loss: 0.6317 - val_acc: 0.6250
Epoch 5/200
640/640 [==============================] - 0s - loss: 0.6468 - acc: 0.5922 - val_loss: 0.6224 - val_acc: 0.6389
Epoch 6/200
640/640 [==============================] - 0s - loss: 0.6376 - acc: 0.5938 - val_loss: 0.6095 - val_acc: 0.6389
Epoch 7/200
640/640 [==============================] - 0s - loss: 0.6264 - acc: 0.6016 - val_loss: 0.5957 - val_acc: 0.6806
Epoch 8/200
640/640 [==============================] - 0s - loss: 0.6133 - acc: 0.6578 

640/640 [==============================] - 0s - loss: 0.4329 - acc: 0.7984 - val_loss: 0.4044 - val_acc: 0.8472
Epoch 67/200
640/640 [==============================] - 0s - loss: 0.4332 - acc: 0.7969 - val_loss: 0.4073 - val_acc: 0.8056
Epoch 68/200
640/640 [==============================] - 0s - loss: 0.4316 - acc: 0.8000 - val_loss: 0.4138 - val_acc: 0.8333
Epoch 69/200
640/640 [==============================] - 0s - loss: 0.4338 - acc: 0.8109 - val_loss: 0.4104 - val_acc: 0.8194
Epoch 70/200
640/640 [==============================] - 0s - loss: 0.4335 - acc: 0.8016 - val_loss: 0.4099 - val_acc: 0.8194
Epoch 71/200
640/640 [==============================] - 0s - loss: 0.4327 - acc: 0.8047 - val_loss: 0.4049 - val_acc: 0.8333
Epoch 72/200
640/640 [==============================] - 0s - loss: 0.4315 - acc: 0.8109 - val_loss: 0.4087 - val_acc: 0.8194
Epoch 73/200
640/640 [==============================] - 0s - loss: 0.4314 - acc: 0.8094 - val_loss: 0.4039 - val_acc: 0.8333
Epoch 74/200


640/640 [==============================] - 0s - loss: 0.4199 - acc: 0.8188 - val_loss: 0.3931 - val_acc: 0.8056
Epoch 132/200
640/640 [==============================] - 0s - loss: 0.4197 - acc: 0.8078 - val_loss: 0.3926 - val_acc: 0.8056
Epoch 133/200
640/640 [==============================] - 0s - loss: 0.4189 - acc: 0.8109 - val_loss: 0.3938 - val_acc: 0.8056
Epoch 134/200
640/640 [==============================] - 0s - loss: 0.4178 - acc: 0.8203 - val_loss: 0.3892 - val_acc: 0.8333
Epoch 135/200
640/640 [==============================] - 0s - loss: 0.4205 - acc: 0.8156 - val_loss: 0.3912 - val_acc: 0.8056
Epoch 136/200
640/640 [==============================] - 0s - loss: 0.4191 - acc: 0.8094 - val_loss: 0.3906 - val_acc: 0.8056
Epoch 137/200
640/640 [==============================] - 0s - loss: 0.4183 - acc: 0.8156 - val_loss: 0.3917 - val_acc: 0.8056
Epoch 138/200
640/640 [==============================] - 0s - loss: 0.4195 - acc: 0.8094 - val_loss: 0.3887 - val_acc: 0.8333
Epoch 

640/640 [==============================] - 0s - loss: 0.4124 - acc: 0.8234 - val_loss: 0.3814 - val_acc: 0.8333
Epoch 197/200
640/640 [==============================] - 0s - loss: 0.4130 - acc: 0.8219 - val_loss: 0.3845 - val_acc: 0.8056
Epoch 198/200
640/640 [==============================] - 0s - loss: 0.4141 - acc: 0.8219 - val_loss: 0.3867 - val_acc: 0.8056
Epoch 199/200
640/640 [==============================] - 0s - loss: 0.4147 - acc: 0.8219 - val_loss: 0.3829 - val_acc: 0.8194
Epoch 200/200
640/640 [==============================] - 0s - loss: 0.4153 - acc: 0.8188 - val_loss: 0.3820 - val_acc: 0.8333


In [8]:
"""
Create submission from model
"""
survived = (model.predict(x_test) > 0.5).astype('int')
passenger_ids = df_test['PassengerId'].values
passenger_ids = passenger_ids.reshape(len(passenger_ids), 1)

print(survived.shape)
print(passenger_ids.shape)

submission = pd.DataFrame(np.hstack([passenger_ids, survived]), columns=['PassengerId', 'Survived'])
submission.to_csv('submission.csv', index=False)
submission.head()

(418, 1)
(418, 1)


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [10]:
model.predict(x_test) > 0.5

array([[False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [